# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colwyn bay,53.2948,-3.7267,12.64,89,99,4.02,GB,1731724459
1,1,kodiak,57.7900,-152.4072,1.73,75,0,6.69,US,1731724205
2,2,el reno,35.5323,-97.9550,14.62,50,0,5.66,US,1731724462
3,3,vadso,70.0744,29.7487,1.09,86,100,4.12,NO,1731724463
4,4,thompson,55.7435,-97.8558,-2.91,93,100,2.57,CA,1731724464


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity', 
    color='City',  # Can be whatever you want
    tiles='OSM',   # Using OpenStreetMap as the tile source
    title='City Humidity Map',
    frame_width=800,  # Set the width of the frame
    frame_height=600  # Set the height of the frame
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
nice_weather_df = city_data_df[(city_data_df['Max Temp']>= 25.3)&(city_data_df['Max Temp']<= 28)].dropna()

# Drop any rows with null values
cleaned_nice_weather_df = nice_weather_df.dropna()

# Display sample data
cleaned_nice_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,west island,-12.1568,96.8225,27.99,83,20,8.23,CC,1731724472
17,17,lihue,21.9789,-159.3672,26.77,71,100,8.23,US,1731724485
25,25,wailua homesteads,22.0669,-159.3780,26.44,77,100,8.23,US,1731724500
43,43,kapa'a,22.0752,-159.3190,27.18,75,100,8.23,US,1731724523
45,45,bonthe,7.5264,-12.5050,25.72,85,64,0.56,SL,1731724525
57,57,nago,26.5881,127.9761,26.73,83,100,3.42,JP,1731724549
79,79,twon-brass,4.3151,6.2418,26.78,79,54,2.57,NG,1731724584
103,103,anloga,5.7947,0.8973,26.72,81,6,2.41,GH,1731724643
107,107,naze,28.3667,129.4833,25.51,87,96,1.61,JP,1731724649
111,111,uturoa,-16.7333,-151.4333,26.80,78,45,5.51,PF,1731724658


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_nice_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = "None"

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,west island,CC,-12.1568,96.8225,83,None
17,lihue,US,21.9789,-159.3672,71,None
25,wailua homesteads,US,22.0669,-159.3780,77,None
43,kapa'a,US,22.0752,-159.3190,75,None
45,bonthe,SL,7.5264,-12.5050,85,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
geoapify_key

'fe54eb7c643b41f3901ea6384fda3412'

In [27]:
# Set parameters to search for a hotel
radius = 10000 #meters
params = {
    "categories": "accommodations.hotel",  # Specify the category for hotels
    "apiKey": geoapify_key, 
    "limit": 5,  # Limit the number of results returned
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']  
    longitude = row['Lng'] 

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Filter for a specific location
    params["bias"] = f"point:{longitude},{latitude}"  # Bias the search towards the hotel's location
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
west island - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
nago - nearest hotel: No hotel found
twon-brass - nearest hotel: No hotel found
anloga - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
papao - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
savannakhet - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
caluula - nearest hotel: No hotel found
pemangkat - nearest hotel: No hotel found
bengkulu - nearest hotel: No hotel found
nanning - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
puerto quijarro - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
lao cai - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
10,west island,CC,-12.1568,96.8225,83,No hotel found
17,lihue,US,21.9789,-159.3672,71,No hotel found
25,wailua homesteads,US,22.0669,-159.3780,77,No hotel found
43,kapa'a,US,22.0752,-159.3190,75,No hotel found
45,bonthe,SL,7.5264,-12.5050,85,No hotel found
57,nago,JP,26.5881,127.9761,83,No hotel found
79,twon-brass,NG,4.3151,6.2418,79,No hotel found
103,anloga,GH,5.7947,0.8973,81,No hotel found
107,naze,JP,28.3667,129.4833,87,No hotel found
111,uturoa,PF,-16.7333,-151.4333,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)